# Daily Email Summarizer — Concierge Agent (Demo)

**Author:** Dev Chauhan

This notebook is a fully runnable demo for the *Agents Intensive Capstone* using a local, reproducible summarizer (no network calls). It demonstrates the workflow of an ADK agent: fetch emails (CSV fallback), summarize, score priority, extract action items, and produce a daily report.

**How to run:** Run all cells. This will create `daily_report.json` and `daily_report.md` in the working directory.

In [ ]:

# Cell 1: Setup - imports and paths
import pandas as pd, json, os, textwrap
from datetime import datetime

DATA_DIR = "."
CSV_PATH = "data/sample_emails.csv"  # relative path when running from repo root / Kaggle notebook
OUT_JSON = "daily_report.json"
OUT_MD = "daily_report.md"


In [ ]:
# Cell 2: Ensure sample data is available in notebook environment (copies provided sample to `data/`)
import os, shutil
os.makedirs("data", exist_ok=True)
shutil.copy("/mnt/data/agents_capstone_email_summarizer/sample_emails.csv", "data/sample_emails.csv")
print("Copied sample_emails.csv -> data/sample_emails.csv")
print("Files in data/:", os.listdir("data"))


In [ ]:

# Cell 3: Load and display sample emails
df = pd.read_csv("data/sample_emails.csv")
df.head(10)


In [ ]:

# Cell 4: Simple deterministic summarizer (mock LLM)
# This function behaves like a small assistant: generates a 1-line summary, priority, reason, and up to 3 actions.
# It uses rule-based heuristics (keywords, sentence splitting) so the demo is fully offline and reproducible.
import re
def simple_summarize(subject, sender, body):
    text = f"{subject}. {body}"
    # extract first meaningful sentence
    sentences = re.split(r'(?<=[.!?])\s+', body.strip())
    first = sentences[0] if sentences and len(sentences[0])>0 else subject
    # create a short summary (max 120 chars)
    summary = (first.strip().replace('\n',' ') )[:120]
    # priority heuristics
    low_keywords = ['welcome','thank you','thanks','invited','webinar']
    high_keywords = ['urgent','asap','immediate','critical','server down','outage','error','failed']
    body_lower = body.lower()
    priority = 'Medium'
    reason = 'Normal message'
    if any(k in body_lower for k in high_keywords) or 'urgent' in subject.lower():
        priority = 'High'
        reason = 'Contains urgent/critical keywords'
    elif any(k in body_lower for k in low_keywords):
        priority = 'Low'
        reason = 'Informational or onboarding message'
    # actions heuristics
    actions = []
    if 'invoice' in subject.lower() or 'invoice' in body_lower:
        actions.append('Review invoice and note due date')
    if 'meeting' in subject.lower() or 'meeting' in body_lower or 'schedule' in body_lower:
        actions.append('Propose new meeting slots')
    if 'verify' in body_lower or 'sign up' in body_lower or 'welcome' in body_lower:
        actions.append('Complete onboarding steps or verify email')
    if 'server' in body_lower or 'outage' in body_lower or 'service is down' in body_lower:
        actions.append('Alert operations team and investigate outage')
    # ensure at least one action
    if not actions:
        actions.append('Read and respond if needed')
    return dict(summary=summary, priority=priority, reason=reason, actions=actions)


In [ ]:

# Cell 5: Agent loop - process emails and generate report
df = pd.read_csv("data/sample_emails.csv")
reports = []
for _, row in df.iterrows():
    res = simple_summarize(row['subject'], row['sender'], str(row['body']))
    reports.append({
        "id": int(row['id']),
        "subject": row['subject'],
        "sender": row['sender'],
        "date": row['date'],
        "summary": res['summary'],
        "priority": res['priority'],
        "reason": res['reason'],
        "actions": res['actions']
    })

# Save JSON
with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(reports, f, ensure_ascii=False, indent=2)
print("Wrote", OUT_JSON, "with", len(reports), "items")


In [ ]:

# Cell 6: Load and display daily report as a DataFrame
import pandas as pd, json
with open(OUT_JSON, "r", encoding="utf-8") as f:
    rpt = json.load(f)

rpt_df = pd.DataFrame(rpt)
rpt_df.head(20)


In [ ]:

# Cell 7: Write a human-friendly daily_report.md
from datetime import datetime
now = datetime.utcnow().isoformat() + "Z"
lines = [f"# Daily Email Summary ({now})\n"]
for item in rpt:
    lines.append(f"## {item['subject']}  
- From: {item['sender']}  
- Date: {item['date']}  
- Priority: **{item['priority']}** — {item['reason']}  \n- Summary: {item['summary']}  \n- Actions:")
    for a in item['actions']:
        lines.append(f"  - {a}")
    lines.append("")

with open(OUT_MD, "w", encoding='utf-8') as f:
    f.write("\n".join(lines))
print('Wrote', OUT_MD)
print('\n---\nSample of markdown:\n') 
print('\n'.join(lines[:18]))


## What to change to use a real inbox (IMAP) or ADK LLM

1. Replace the `simple_summarize` function with a call to your ADK agent or LLM. Example pseudocode:

```python
from adk import AgentClient
agent = AgentClient(...)
resp = agent.call(prompt)
# parse JSON output
```

2. For live email fetching, implement `imaplib` usage and read credentials from a secure location (do NOT commit credentials). On Kaggle use CSV fallback as done here.

---

### Files generated by this notebook
- `daily_report.json` — structured JSON with summary, priority, actions (for automation)
- `daily_report.md` — human-readable markdown summary

Run all cells, then download the files from the notebook environment (Kaggle/Colab) or from the repository folder.